### Query Decomposition:

Query decomposition is the process of takinga complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

It is useful in
* Complex queries often involve multiple concepts.
* LLMs or retrievers may miss parts of the original question.
* It enables multi-hop reasoning (answering in steps)
* Allows parallelism(especially in multi-agent frameworks)

In [2]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

d:\RAGs In Depth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Data Ingestion and Embedding Documents

documents = TextLoader('langchain_crewai.txt', encoding="utf-8").load()

chunks = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50).split_documents(documents)

embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

vectorstore= FAISS.from_documents(chunks, embeddings)

retriever= vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":4, "lambda_mult":0.7})

#The lambda_mult parameter controls this trade-off:

# lambda_mult = 1.0 → prioritize only relevance (like normal similarity search)
# lambda_mult = 0.0 → prioritize only diversity

In [4]:
#llm model
llm = init_chat_model(model="groq:llama-3.1-8b-instant")
llm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001C9192FAF10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C96CE3CD50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
#query decomposition prompt

decompose_prompt = PromptTemplate.from_template(
    '''
    You are an AI Assistant. Decompose the following complex question into 2 or more sub-questions for better document retrieval.
    question: {question}

    sub-question: 
    '''
)

decomposition_chain = decompose_prompt | llm | StrOutputParser()

decomposition_chain

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n    You are an AI Assistant. Decompose the following complex question into 2 or more sub-questions for better document retrieval.\n    question: {question}\n\n    sub-question: \n    ')
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001C9192FAF10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C96CE3CD50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [6]:
query = "How does langchain use memory and agents compared to crewai?"
sub_questions= decomposition_chain.invoke({"question": query})
sub_questions

"To decompose the complex question into sub-questions for better document retrieval, I would suggest the following:\n\n1. **Langchain architecture**: What are the key components of the Langchain memory system?\n2. **Langchain agents**: How do Langchain agents interact with the memory system to generate responses?\n3. **CrewAI overview**: What are the primary differences between Langchain and CrewAI in terms of their architecture and approach?\n4. **Memory usage comparison**: How does Langchain's memory usage compare to CrewAI's in terms of storage and retrieval?\n5. **Agent comparison**: What are the key differences between Langchain's agents and CrewAI's agents in terms of their capabilities and functionality?\n6. **Langchain vs CrewAI**: What are the primary advantages and disadvantages of Langchain's memory and agent systems compared to CrewAI's?\n\nBy breaking down the complex question into these sub-questions, we can retrieve more specific and relevant information from documents, 

In [7]:
print(sub_questions)

To decompose the complex question into sub-questions for better document retrieval, I would suggest the following:

1. **Langchain architecture**: What are the key components of the Langchain memory system?
2. **Langchain agents**: How do Langchain agents interact with the memory system to generate responses?
3. **CrewAI overview**: What are the primary differences between Langchain and CrewAI in terms of their architecture and approach?
4. **Memory usage comparison**: How does Langchain's memory usage compare to CrewAI's in terms of storage and retrieval?
5. **Agent comparison**: What are the key differences between Langchain's agents and CrewAI's agents in terms of their capabilities and functionality?
6. **Langchain vs CrewAI**: What are the primary advantages and disadvantages of Langchain's memory and agent systems compared to CrewAI's?

By breaking down the complex question into these sub-questions, we can retrieve more specific and relevant information from documents, ultimately

In [8]:
#QA chain per sub_question

qa_prompt = PromptTemplate.from_template(
    ''' Use the context belwo to answer the question.

    context: {context}

    question: {input}
    '''
)

documents_chain = create_stuff_documents_chain(llm, qa_prompt)

In [27]:
#getting each question from sub_questions
import re
questions=sub_questions.split("\n\n")
question=questions[1].split("\n")
u=[]
for q in question:
    q = re.sub(r"\d+\.\s*\*\*.*?\*\*:", "", q)
    u.append(q)

In [36]:
from langchain_core.runnables import RunnablePassthrough
for q in u:
    rag_chain = (
        {"context": retriever, "input": RunnablePassthrough()}
        | qa_prompt
        | llm
        | StrOutputParser())
    response = rag_chain.invoke(q)
    print(f"Question: {q}")
    print(response, "\n", "*"*80)





Question:  What are the key components of the Langchain memory system?
Based on the provided context, the key components of the Langchain memory system mentioned are:

1. **Prompt templates**: These are likely pre-defined or customizable templates for generating prompts to be used by the Langchain system.
2. **Memory**: This is a component that allows agents to maintain continuity and reuse information, which implies persisting data across interactions or sessions.

These components are mentioned in the document with the id 'a58203ca-bd87-4d5f-aa43-524342101bb1', specifically in the page content 'LangChain’s support for prompt templates and memory means that agents can maintain continuity and reuse'. 
 ********************************************************************************
Question:  How do Langchain agents interact with the memory system to generate responses?
Based on the provided context, the interaction between LangChain agents and the memory system is not explicitly menti